In [ ]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 11.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://unnecessitated-kendall-gogetting.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://unnecessitated-kendall-gogetting.ngrok-free.dev


True

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [ ]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology），簡稱明新科大，是一所位於台灣新竹縣新豐鄉的私立科技大學。學校創立於1966年，前身為「明新工業專科學校」。

**發展沿革**
明新科大於1966年3月1日以工專立案，初期設有機械、土木、工業管理三科。隨著國家經濟發展和產業需求，陸續增設電機工程科、化學工程科等。1993年更名為「私立明新工商專科學校」。1997年7月，獲教育部核准改制為「明新技術學院」並附設專科部。最終在2002年8月，明新獲教育部核准優先升格為「明新科技大學」。2018年12月，學校更名為「明新學校財團法人明新科技大學」。

**辦學理念與特色**
明新科技大學秉持「堅毅、求新、創造」的校訓，並以《大學》中「在明明德，在新民，在止於至善」的精義為校名精神，旨在培育學生高尚品德、專業學問與優良技術，以期達到全人發展的目標。學校明確定位為「一流產業大學」，致力於培育「跨域整合、務實創新與全人學習」的專業人才。

**學術單位**
目前明新科技大學設有六大學院，包括：半導體學院、工程學院、管理學院、民生學院、人文與設計學院、共同教育學院。共涵蓋20個學系、2個學位學程（含1個博士學位學程）及11個碩士班。其中，半導體學院的成立，整合了電機工程系、電子工程系、化學工程與材料科技系、光電工程系，致力於半導體人才的培育。學校也設有全國第一個「風力發電學士學位學程」，並積極推動AI教育與產學合作。

**重要成就與榮譽**
明新科技大學深耕在地，與新竹工業區及科學園區緊密合作，為區域產業培養大量人才，被譽為新竹縣人才供應的第一品牌。在2022年《遠見》雜誌「企業最愛公私立技職科大調查」中，明新科大在「四大領域」獲得兩項企業最愛，起薪排名私校第一，甚至優於部分國立科大。根據1111人力銀行統計，半導體產業界最愛聘用的畢業生中，明新科大名列第四，是唯一入榜的私立科大，與台、清、交、成等頂尖大學並列前五大。此外，學校也榮獲教育部全國技專校院勞作教育及服務學習績優學校第一名，以及教育部青年發展署服務學習示範性計畫大專校院組特優等榮譽。工學院多個系所也通過中華工程教育學會（IEET）的工程認證，確保學生具備國際認可的專業核心能力。


In [ ]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學的現任校長是**呂明峯**教授。他於2025年2月1日正式上任，成為明新科大的第11任校長。

呂明峯校長在明新科技大學任職34年，擁有豐富的業界經驗和學術背景，並曾打造全台首座半導體封裝測試類產線，推動成立半導體學院。 他提出「四大核心模組」作為校務治理策略，致力於將明新科大打造成為「具國際魅力的產業科技大學」。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit


BODY:  {"destination":"U4266981d0cd4ada66e0c3da0512201fb","events":[{"type":"message","message":{"type":"text","id":"592601164122685441","quoteToken":"mi55reDwSwSble5NzAq6CBXWry-rLiTV9eGRCp2N-dugvLeKml4bkncDsUqzdK-mt2kyxJUhSBcbFFh3r_YosKxOj0J7lpVLTRUYH26hvFZm77WrMorn6RG1XJizO84rPbDuSx-mhKpyD1d0sFYSog","markAsReadToken":"7pdIoNIMooJzMO0rPD_YVhBXktzzkcCUm-Blpczp4GnLY7TX4Y6RJ-lR7boolbj9JkEJJbh1tf__MHxuiWlHnG09hQXW5-D_4HzXqR-yVwH1bACixJb9j7k7SN8gQEp2PYzhJrFwF7XAHLQqxG5uJuVlV_4RaFj3smttyp-__jGS89ckEg97xzqlqtQRH_U2lR15xUvmiQhV4kx6r5qAnQ","text":"AI 介紹一下明新科大(五十字以內)"},"webhookEventId":"01KCRBP2P6AW3B1JRGA6747GSK","deliveryContext":{"isRedelivery":false},"timestamp":1766049122818,"source":{"type":"user","userId":"U0eb3bfa63bf656ad470ba1f81015848e"},"replyToken":"d7b4b67ef673425e98c67404437b5114","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [18/Dec/2025 09:12:05] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U4266981d0cd4ada66e0c3da0512201fb","events":[{"type":"message","message":{"type":"text","id":"592601186755674867","quoteToken":"4Rf1M_qjIOtgXwu7DyQUYq3MOowEEQze3QPfKOzUPNBRp-meNUCPhWangZxcsxPXDx3-tBDyNhCFQ5pRUjy59aga3S1-qO9bAwM1iAuQJR0-30UrMeVJYpFtOv4TAc4aXIE9cfko__8TYKJoKJxG8w","markAsReadToken":"_ac1BZ8mwkeXkQYBVPRKVd5unoYnX1k089PpKVI5jIEMHFS2ME_hRsyzSlkQIgtfCLB-ewgvpnkk9Jt5WPi6Q9vsfr0UvaJr3ojU7Q8uiLViEY0qhrdqQW2BZ63XYrqoNoRuGU98TlpcQx1RDjl15EaLfw2VEzkVmezl2tip4Ep-IBzYTG8m0wGI2OwvMzr3zvrHOrwojha8a8Wdp_rs1g","text":"AI 校長是誰?"},"webhookEventId":"01KCRBPG7439DA1KS09N1H54T2","deliveryContext":{"isRedelivery":false},"timestamp":1766049136512,"source":{"type":"user","userId":"U0eb3bfa63bf656ad470ba1f81015848e"},"replyToken":"0703b0076db742b68f46dd13ec81e6c9","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [18/Dec/2025 09:12:18] "POST / HTTP/1.1" 200 -
